In [10]:
import sys
import os

# 获取当前工作目录（用于替代 __file__）
current_dir = os.getcwd()

target_path1 = os.path.abspath(os.path.join(current_dir, '../3_move'))
target_path2 = os.path.abspath(os.path.join(current_dir, '../1_getPointCloud'))

# 将路径添加到 sys.path
if target_path1 not in sys.path:
    sys.path.append(target_path1)
    
if target_path2 not in sys.path:
    sys.path.append(target_path2)
    
# 检查路径是否添加成功
print("Current Python Path:")
print("\n".join(sys.path))

from ImageRecognizer import ImageRecognizer
image_recognizer = ImageRecognizer(top_dir="/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode/2_perception/cubes/")


Current Python Path:

/opt/ros_ws/devel/lib/python3/dist-packages
/opt/ros/noetic/lib/python3/dist-packages
/usr/lib/python38.zip
/usr/lib/python3.8
/usr/lib/python3.8/lib-dynload
/usr/local/lib/python3.8/dist-packages
/usr/lib/python3/dist-packages
/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode/3_move
/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode/1_getPointCloud


In [11]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def matrix_to_rpy_and_translation(matrix):
    """
    从4x4变换矩阵中提取RPY（Roll, Pitch, Yaw）和位移信息，并返回一个列表形式。
    
    参数:
        matrix: 4x4变换矩阵 (numpy.ndarray)
        
    返回:
        result: 一个包含 [roll, pitch, yaw, x, y, z] 的列表
    """
    if matrix.shape != (4, 4):
        raise ValueError("输入必须是一个 4x4 矩阵")
    
    # 提取旋转矩阵和位移
    rotation_matrix = matrix[:3, :3]
    translation = matrix[:3, 3]
    
    # 创建旋转矩阵的副本，避免只读内存问题
    rotation_matrix = np.array(rotation_matrix)
    
    # 使用scipy转换为RPY角
    r = R.from_matrix(rotation_matrix)
    roll, pitch, yaw = r.as_euler('xyz', degrees=False)
    
    # 返回结果
    return [roll, pitch, yaw], translation.tolist()


In [12]:
from save_point_cloud import PointCloudSaver
import open3d as o3d
import rospy
point_cloud_saver = PointCloudSaver()

# 等待数据准备
rospy.loginfo("Waiting for data...")
rospy.sleep(1)  # 等待话题数据发布

# 保存点云
world_file = "/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode/2_perception/mesh/zed_point_cloud_world3.ply"
point_cloud_saver.save_point_clouds(world_file)


from PickAndPlace import PickAndPlace
pick_place = PickAndPlace(approach_distance=0.3)


INFO - 2025-01-31 07:12:13,807 - topics - topicmanager initialized


[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m


[%t] [%l] %m


[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m


In [13]:
import open3d as o3d
import numpy as np

def filter_point_cloud_by_depth_and_range(point_cloud, depth_threshold=0.005, range=[0.001, -0.8, 1, 1.6]):
    """
    读取点云文件并过滤掉深度低于指定阈值和超出检测范围的点。

    Args:
        point_cloud (o3d.geometry.PointCloud): 点云对象。
        depth_threshold (float): 深度阈值，低于该值的点将被过滤。
        range (list): 过滤范围，包含 [x, y, w, h]。
        
    Returns:
        o3d.geometry.PointCloud: 过滤后的点云对象。
    """

    # 获取点云的点坐标
    points = np.asarray(point_cloud.points)

    # 过滤深度小于阈值的点
    filtered_points = points[points[:, 2] >= depth_threshold]

    # 根据范围过滤点 (x, y, w, h) -> 左上角和宽高
    x, y, w, h = range
    filtered_points = filtered_points[
        (filtered_points[:, 0] >= x) & (filtered_points[:, 0] <= x + w) &
        (filtered_points[:, 1] >= y) & (filtered_points[:, 1] <= y + h)
    ]

    # 创建新的点云对象
    filtered_point_cloud = o3d.geometry.PointCloud()
    filtered_point_cloud.points = o3d.utility.Vector3dVector(filtered_points)

    # 保留颜色信息（如果存在）
    if point_cloud.has_colors():
        colors = np.asarray(point_cloud.colors)
        filtered_colors = colors[points[:, 2] >= depth_threshold]
        filtered_point_cloud.colors = o3d.utility.Vector3dVector(filtered_colors)

    return filtered_point_cloud

def filter_point_cloud_by_depth(point_cloud, depth_threshold=0.005):
    """
    读取点云文件并过滤掉深度低于指定阈值的点。

    Args:
        ply_path (str): 点云文件的路径。
        depth_threshold (float): 深度阈值，低于该值的点将被过滤。

    Returns:
        o3d.geometry.PointCloud: 过滤后的点云对象。
    """


    # 获取点云的点坐标
    points = np.asarray(point_cloud.points)

    # 过滤深度小于阈值的点
    filtered_points = points[points[:, 2] >= depth_threshold]

    # 创建新的点云对象
    filtered_point_cloud = o3d.geometry.PointCloud()
    filtered_point_cloud.points = o3d.utility.Vector3dVector(filtered_points)

    # 保留颜色信息（如果存在）
    if point_cloud.has_colors():
        colors = np.asarray(point_cloud.colors)
        filtered_colors = colors[points[:, 2] >= depth_threshold]
        filtered_point_cloud.colors = o3d.utility.Vector3dVector(filtered_colors)

    return filtered_point_cloud

zed_ply_path = "mesh/zed_point_cloud_world3.ply"

coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1,  # 坐标轴大小，可以根据需要调整
    origin=[0, 0, 0]  # 坐标轴的原点
)

# 读取点云文件
point_cloud = o3d.io.read_point_cloud(zed_ply_path)
if not point_cloud.has_points():
    raise ValueError(f"Failed to read point cloud from {zed_ply_path}")
filtered_point_cloud = filter_point_cloud_by_depth_and_range(point_cloud, depth_threshold=0.005, range=[0.001, -0.8, 1, 1.6])
o3d.visualization.draw_geometries([filtered_point_cloud, coordinate_frame], window_name="Filtered Point Cloud")


In [14]:
def calculate_max_layer(filtered_point_cloud, layer_height=0.04):
    """
    计算点云的最高层数 (MaxLayer)，基于 z 轴最大值，使用四舍五入计算。

    Args:
        filtered_point_cloud (o3d.geometry.PointCloud): 已过滤的点云对象。
        layer_height (float): 每一层的高度，默认值为 0.04。

    Returns:
        int: 最高层数 (MaxLayer)。
    """
    # 提取点云的 z 轴最大值
    points = np.asarray(filtered_point_cloud.points)
    z_max = np.max(points[:, 2])

    # 计算最高层数，严格四舍五入
    max_layer = int(np.floor(z_max / layer_height + 0.5))

    return max_layer

# 示例调用
max_layer = calculate_max_layer(filtered_point_cloud, layer_height=0.04)
print(f"MaxLayer: {max_layer}")


MaxLayer: 1


In [15]:
import copy
import open3d as o3d
import numpy as np

def register_and_filter(pointcloud, mesh, voxel_size=0.01):
    # 转换mesh为点云
    if isinstance(mesh, o3d.geometry.TriangleMesh):
        mesh_pointcloud = mesh.sample_points_uniformly(number_of_points=1000)
    elif isinstance(mesh, o3d.geometry.PointCloud):
        mesh_pointcloud = copy.deepcopy(mesh)
    
    # 下采样点云并计算特征
    def preprocess_point_cloud(pcd, voxel_size):
        pcd_down = pcd.voxel_down_sample(voxel_size)
        pcd_down.estimate_normals(
            search_param=o3d.geometry.KDTreeSearchParamHybrid(
                radius=voxel_size*2, 
                max_nn=30
            )
        )
        pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
            pcd_down,
            search_param=o3d.geometry.KDTreeSearchParamHybrid(
                radius=voxel_size*5, 
                max_nn=100
            )
        )
        return pcd_down, pcd_fpfh

    # 粗配准
    def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
        distance_threshold = voxel_size * 1.5
        result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            source_down, target_down, source_fpfh, target_fpfh, True,
            distance_threshold,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
            4,
            [
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
            ],
            o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
        )
        return result

    # 精配准
    def refine_registration(source, target, initial_transformation, voxel_size):
        distance_threshold = voxel_size * 1  # 减小阈值提高精度
        result = o3d.pipelines.registration.registration_icp(
            source, target, distance_threshold, initial_transformation,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000000)
        )
        return result

    # 执行点云预处理
    source_down, source_fpfh = preprocess_point_cloud(mesh_pointcloud, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(pointcloud, voxel_size)

    # 执行配准
    coarse_result = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
    refined_result = refine_registration(mesh_pointcloud, pointcloud, coarse_result.transformation, voxel_size)

    # 变换网格点云
    transform = refined_result.transformation
    transformed_mesh_pointcloud = mesh_pointcloud.transform(transform)

    # 创建包围盒
    oriented_bounding_box = transformed_mesh_pointcloud.get_oriented_bounding_box()
    center = oriented_bounding_box.center
    extent = oriented_bounding_box.extent
    rotation_matrix = oriented_bounding_box.R

    # 扩展包围盒
    margin = voxel_size
    expanded_extent = extent + 1.0 * margin
    expanded_bounding_box = o3d.geometry.OrientedBoundingBox(
        center=center,
        extent=expanded_extent,
        R=rotation_matrix
    )

    # 过滤点云
    indices_inside_box = expanded_bounding_box.get_point_indices_within_bounding_box(pointcloud.points)
    indices_outside_box = list(set(range(len(pointcloud.points))) - set(indices_inside_box))

    # 分离点云
    remaining_pointcloud = pointcloud.select_by_index(indices_outside_box)
    deleted_pointcloud = pointcloud.select_by_index(indices_inside_box)

    return transform, remaining_pointcloud, deleted_pointcloud, refined_result.fitness

In [16]:
# 读取cube
# 定义文件路径
cube_obj_path = "mesh/cube_0.obj"
zed_ply_path = "mesh/zed_point_cloud_world3.ply"

# 读取 cube_0.obj 点云
cube_mesh = o3d.io.read_triangle_mesh(cube_obj_path)
cube_mesh.compute_vertex_normals()  # 计算法线以便更好显示
# cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=50000)  # 转为点云
# cube_point_cloud = cube_mesh.sample_points_poisson_disk(number_of_points=1000)

# 去掉cube下半部分，防止z轴翻转
# cube_point_cloud = filter_point_cloud_by_depth(cube_point_cloud, depth_threshold=-0.015)
# o3d.visualization.draw_geometries([cube_point_cloud])


coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1,  # 坐标轴大小，可以根据需要调整
    origin=[0, 0, 0]  # 坐标轴的原点
)


In [17]:
def check_transform_z_axis_alignment(transform, tolerance=0.1):
    """
    检查 transform 的 Z 轴是否与世界坐标系的 Z 轴平行。
    允许一定的容差范围内，判断是否平行或反向平行。
    参数:
    - transform: 4x4 转换矩阵
    - tolerance: 判断的容差范围，默认 0.1
    返回:
    - True: 如果 Z 轴平行或反向平行
    - False: 如果 Z 轴不平行
    - 修正后的
    """
    z_axis = np.array([0, 0, 1])  # 世界坐标系的 Z 轴
    transform_z_axis = transform[:3, 2]  # 获取 transform 的 Z 轴（即旋转矩阵的第三列）

    # 计算 transform Z 轴与世界坐标系 Z 轴的夹角
    dot_product = np.dot(transform_z_axis, z_axis)
    # 计算夹角的余弦值，如果接近 1 或 -1，表示平行或反向平行
    if np.abs(dot_product) > (1 - tolerance):
        return True
    return False

def align_transform_z_axis(transform):
    """
    如果 transform 的 Z 轴与世界坐标系 Z 轴反向平行，
    则通过绕 X 轴旋转 180 度（np.pi）来翻转 Z 轴方向。
    
    参数:
    - transform: 4x4 转换矩阵
    
    返回:
    - 修正后的 transform 矩阵
    """
    z_axis_world = np.array([0, 0, 1])  # 世界坐标系的 Z 轴
    transform_z_axis = transform[:3, 2]  # 获取 transform 的 Z 轴（旋转矩阵的第三列）

    # 判断 Z 轴是否与世界坐标系 Z 轴反向平行
    if np.dot(transform_z_axis, z_axis_world) < 0:  # Z 轴反向
        print("修正z轴")
        # 创建绕 X 轴旋转 180 度的旋转矩阵
        rotation_matrix = np.eye(4)
        rotation_matrix[1, 1] = -1  # 旋转矩阵绕 X 轴旋转 180 度
        rotation_matrix[2, 2] = -1  # 旋转矩阵绕 X 轴旋转 180 度
        
        # 进行矩阵乘法，旋转矩阵应用到原始 transform 上
        transform = np.dot(rotation_matrix, transform)

    return transform

In [18]:
import numpy as np
import open3d as o3d
import numpy as np
from scipy.ndimage import gaussian_filter

def max_downsample_image(image, pool_size=(2, 2)):
    """
    使用最大值池化对图像进行降采样。

    Args:
        image (numpy.ndarray): 输入彩色图像 (height, width, 3)
        pool_size (tuple): 池化窗口大小 (height, width)

    Returns:
        downsampled_image (numpy.ndarray): 降采样后的图像
    """
    h, w, c = image.shape
    ph, pw = pool_size

    # 确保图像尺寸是池化窗口的整数倍
    h_new = h // ph * ph
    w_new = w // pw * pw
    image_cropped = image[:h_new, :w_new, :]

    # 使用最大池化降采样
    downsampled_image = image_cropped.reshape(h_new // ph, ph, w_new // pw, pw, c).max(axis=(1, 3))

    return downsampled_image

def pointcloud_to_top_view_image_color(pointcloud, voxel_size=0.01, output_image_size=(512, 512)):
    """
    从点云生成正上方的彩色 2D 图像。
    
    Args:
        pointcloud (o3d.geometry.PointCloud): 输入点云
        voxel_size (float): 分辨率，用于划分网格
        output_image_size (tuple): 输出图像大小 (width, height)
    
    Returns:
        top_view_image (numpy.ndarray): 生成的 2D 彩色图像
    """
    # 获取点云的点坐标和颜色
    points = np.asarray(pointcloud.points)
    colors = np.asarray(pointcloud.colors) if pointcloud.has_colors() else np.zeros_like(points)

    # 提取 x-y 范围
    x_min, y_min = np.min(points[:, :2], axis=0)
    x_max, y_max = np.max(points[:, :2], axis=0)
    print(x_max, y_max)

    # 定义网格分辨率
    width, height = output_image_size
    grid_x = np.linspace(x_min, x_max, width)
    grid_y = np.linspace(y_min, y_max, height)

    # 初始化 2D 彩色图像
    top_view_image = np.zeros((height, width, 3))  # RGB 图像

    # 初始化计数矩阵以累积颜色
    count_matrix = np.zeros((height, width))

    # 遍历点云，将每个点投影到网格上
    for point, color in zip(points, colors):
        x, y, z = point
        # 找到对应的网格索引
        x_idx = int((x - x_min) / (x_max - x_min) * (width - 1))
        y_idx = int((y - y_min) / (y_max - y_min) * (height - 1))
        
        # 更新彩色图像和计数矩阵
        top_view_image[height - 1 - y_idx, x_idx] += color  # 累加颜色
        count_matrix[height - 1 - y_idx, x_idx] += 1

    # 平均化颜色值
    nonzero_mask = count_matrix > 0
    top_view_image[nonzero_mask] /= count_matrix[nonzero_mask, None]

    # 将结果归一化到 0-1 范围内
    top_view_image = np.clip(top_view_image, 0, 1)

    return top_view_image


import scipy.ndimage

def interpolate_sparse_image(image, dilation_size=(7, 7)):
    """
    对稀疏的彩色图像进行插值填充。
    
    Args:
        image (numpy.ndarray): 输入彩色图像 (height, width, 3)
        dilation_size (tuple): 膨胀操作的邻域大小 (height, width)
    
    Returns:
        interpolated_image (numpy.ndarray): 插值后的彩色图像
    """
    interpolated_image = image.copy()
    for i in range(3):  # 分别处理 RGB 三个通道
        interpolated_image[:, :, i] = scipy.ndimage.morphology.grey_dilation(
            image[:, :, i], size=dilation_size
        )
    return interpolated_image


def pointcloud_to_colored_image_with_filling(pointcloud, output_size=(512, 512), voxel_size=0.01):
    """
    从点云生成正上方的彩色 2D 图像，使用高斯滤波填充空白区域。
    
    Args:
        pointcloud (o3d.geometry.PointCloud): 输入点云。
        output_size (tuple): 输出图像大小 (width, height)。
        voxel_size (float): 网格分辨率，用于划分网格。
    
    Returns:
        top_view_image (numpy.ndarray): 填充后的彩色图像。
    """
    # 获取点云的点坐标和颜色
    points = np.asarray(pointcloud.points)
    colors = np.asarray(pointcloud.colors) if pointcloud.has_colors() else np.zeros_like(points)

    # 提取 x-y 范围
    x_min, y_min = np.min(points[:, :2], axis=0)
    x_max, y_max = np.max(points[:, :2], axis=0)

    # 定义网格分辨率
    width, height = output_size
    grid_x = np.linspace(x_min, x_max, width)
    grid_y = np.linspace(y_min, y_max, height)

    # 初始化 2D 彩色图像和计数矩阵
    top_view_image = np.zeros((height, width, 3), dtype=np.float32)  # RGB 图像
    count_matrix = np.zeros((height, width), dtype=np.float32)

    # 遍历点云，将每个点投影到网格上
    for point, color in zip(points, colors):
        x, y, z = point
        # 找到对应的网格索引
        x_idx = int((x - x_min) / (x_max - x_min) * (width - 1))
        y_idx = int((y - y_min) / (y_max - y_min) * (height - 1))
        
        # 累积颜色值
        top_view_image[height - 1 - y_idx, x_idx] += color  # 累加颜色
        count_matrix[height - 1 - y_idx, x_idx] += 1

    # 归一化颜色值
    nonzero_mask = count_matrix > 0
    top_view_image[nonzero_mask] /= count_matrix[nonzero_mask, None]

    # 使用高斯滤波填充空白区域
    for channel in range(3):  # 对 R/G/B 通道分别处理
        top_view_image[:, :, channel] = gaussian_filter(top_view_image[:, :, channel], sigma=2)

    # 将图像裁剪到 [0, 1] 范围
    top_view_image = np.clip(top_view_image, 0, 1)

    return top_view_image

def triangle_mesh_to_image(mesh, image_size=(100, 100), fill_radius=1):
    """
    将 TriangleMesh 的顶点投影到 2D 图像上，并通过最大池化生成较小的图像。
    确保每个像素点有值。
    
    Args:
        mesh (o3d.geometry.TriangleMesh): 输入三角网格。
        image_size (tuple): 输出图像大小 (width, height)。
        pool_size (tuple): 池化窗口大小 (height, width)。
        fill_radius (int): 每个点影响的邻域半径（像素）。
    
    Returns:
        numpy.ndarray: 投影后的 2D 图像（池化后大小）。
    """
    # 获取顶点和颜色
    vertices = np.asarray(mesh.vertices)
    colors = np.asarray(mesh.vertex_colors) if mesh.has_vertex_colors() else np.ones((len(vertices), 3))

    # 提取 x-y 范围
    x_min, y_min = vertices[:, 0].min(), vertices[:, 1].min()
    x_max, y_max = vertices[:, 0].max(), vertices[:, 1].max()

    # 初始化 2D 图像
    width, height = image_size
    image = np.zeros((height, width, 3), dtype=np.float32)
    count_matrix = np.zeros((height, width), dtype=np.int32)  # 用于统计像素值累计次数

    # 映射顶点到图像坐标并填充周围像素
    for vertex, color in zip(vertices, colors):
        x, y = vertex[0], vertex[1]
        u = int((x - x_min) / (x_max - x_min) * (width - 1))
        v = int((y - y_min) / (y_max - y_min) * (height - 1))

        # 填充当前点和周围像素
        for du in range(-fill_radius, fill_radius + 1):
            for dv in range(-fill_radius, fill_radius + 1):
                uu = min(max(u + du, 0), width - 1)
                vv = min(max(v + dv, 0), height - 1)
                image[vv, uu] += color
                count_matrix[vv, uu] += 1

    # 平均化每个像素的颜色值
    mask = count_matrix > 0
    image[mask] /= count_matrix[mask, None]

    # 将图像转换为 0-255 的范围
    image = (np.clip(image, 0, 1) * 255).astype(np.uint8)

    return image

def enlarge_points_as_cubes(point_cloud, cube_size=0.0001):
    """
    将点云的每个点替换为一个立方体以增大显示大小。
    
    Args:
        point_cloud (o3d.geometry.PointCloud): 输入点云。
        cube_size (float): 立方体的边长。
    
    Returns:
        o3d.geometry.TriangleMesh: 包含所有点的立方体的组合几何体。
    """
    points = np.asarray(point_cloud.points)
    colors = np.asarray(point_cloud.colors) if point_cloud.has_colors() else [[1, 0, 0]] * len(points)
    mesh = o3d.geometry.TriangleMesh()

    for point, color in zip(points, colors):
        # 创建一个小立方体并移动到点的位置
        cube = o3d.geometry.TriangleMesh.create_box(width=cube_size, height=cube_size, depth=cube_size)
        cube.translate(point - np.array([cube_size / 2, cube_size / 2, cube_size / 2]))
        cube.paint_uniform_color(color)
        mesh += cube  # 将立方体添加到组合网格中

    return mesh


In [19]:
from openai import OpenAI
from io import BytesIO
from PIL import Image
import base64
import json
SYSTEM_PROMPT = """Please act as an image recognition agent. 
You will be given a square face of a block, 
which is projected from a point cloud. 
Your task is to recognize the following:

Determine if this is a block face.
Each face contains only one letter, 
one pattern (just detect whether it's a pattern, no need to identify the exact pattern), 
or is blank (only wood texture). 
Please detect whether it is a letter, 
a pattern, or blank. 
Each of these may be rotated. 
Please analyze all possible rotations in a clockwise direction: 0°, 90°, 180°, and 270°.
There might be a circular border around the face. 
Please detect if this border exists. 
It's confirmed that the color of the border matches the color of the letter or pattern.
The expected output is a JSON in the following format:
{
    "check": true/false, 
    "c": char/"pattern"/"blank", 
    "color": "green"/"yellow"/"red"/"blue"/"None", 
    "rotation": 0/90/180/270, 
    "circle": true/false
}
"""

api_key=""
client = OpenAI(api_key=api_key)

def encode_image(image, quality=100):
    if image.mode != 'RGB':
        image = image.convert('RGB')  # Convert to RGB
    buffered = BytesIO()
    image.save(buffered, format="JPEG", quality=quality) 
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

def gpt4o_analysis(image_path, quality=50):
    with Image.open(image_path) as img:
        img_b64_str = encode_image(img, quality=quality)
    img_type = "image/jpeg"
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": SYSTEM_PROMPT},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:{img_type};base64,{img_b64_str}"},
                    },
                ],
            }
        ],
    )
    return response.choices[0].message.content

In [20]:
import rospy
import tf
import numpy as np
from geometry_msgs.msg import TransformStamped

class TransformBroadcaster:
    def __init__(self):
        # 尝试初始化ROS节点，避免多次初始化
        try:
            rospy.init_node('tf_broadcaster_node')
        except rospy.exceptions.ROSException:
            pass  # 如果节点已初始化，就不做任何操作

        # 创建一个TransformBroadcaster实例
        self.br = tf.TransformBroadcaster()

        # 假设T是给定的4x4变换矩阵
        self.T = np.ones((4, 4))  # 修改为4x4矩阵

        # 设置定时器，每100毫秒调用一次broadcast_transform函数
        self.timer = rospy.Timer(rospy.Duration(0.1), self.broadcast_transform)

    def broadcast_transform(self, event):
        # 从4x4矩阵中提取旋转部分和位移部分
        translation = self.T[0:3, 3]  # 位移部分 (x, y, z)
        rotation_matrix = self.T[0:3, 0:3]  # 旋转矩阵部分

        # 创建一个完整的4x4矩阵，包括旋转矩阵和齐次坐标
        full_matrix = np.eye(4)
        full_matrix[0:3, 0:3] = rotation_matrix
        full_matrix[0:3, 3] = translation

        # 创建一个Quaternion（四元数）来表示旋转
        quaternion = tf.transformations.quaternion_from_matrix(full_matrix)

        # 发布变换
        self.br.sendTransform(
            (translation[0], translation[1], translation[2]),  # 位移部分
            (quaternion[0], quaternion[1], quaternion[2], quaternion[3]),  # 旋转部分（四元数）
            rospy.Time.now(),  # 时间戳
            "cube",  # 子坐标系名称
            "world"   # 父坐标系名称
        )

    def update(self, T):
        self.T = T
        
    def stop(self):
        # 停止定时器
        self.timer.shutdown()


In [21]:
from utils import *  # 假设你的create_grasp_mesh函数在utils.py中
import numpy as np
import open3d as o3d

def generate_gripper_from_transform(T: np.ndarray):
    """
    Generates a robotic gripper mesh from a given 4x4 transformation matrix,
    with additional rotations around x and y axes.

    Args:
        T: 4x4 transformation matrix (numpy array).
        
    Returns:
        gripper_meshes: List of meshes representing the gripper.
    """
    # 提取旋转矩阵（3x3）
    rotation_matrix = T[:3, :3]

    # 提取平移向量
    translation = T[:3, 3]

    # 设置抓手的中心点位置，通常为平移向量
    center_point = translation

    # 创建绕x轴旋转 -90 度的旋转矩阵
    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(-np.pi/2), -np.sin(-np.pi/2)],
        [0, np.sin(-np.pi/2), np.cos(-np.pi/2)]
    ])

    # 创建绕y轴旋转 90 度的旋转矩阵
    R_y = np.array([
        [np.cos(np.pi/2), 0, np.sin(np.pi/2)],
        [0, 1, 0],
        [-np.sin(np.pi/2), 0, np.cos(np.pi/2)]
    ])
    
    R_z = np.array([
        [0, -1, 0],
        [1, 0, 0],
        [0, 0, 1]
    ])
    # 组合旋转矩阵，先绕 x 轴旋转，再绕 y 轴旋转
    combined_rotation = R_z @ rotation_matrix @ R_x 

    # 调用 create_grasp_mesh 函数生成抓手
    gripper_meshes = create_grasp_mesh(
        center_point=center_point, 
        rotation_matrix=combined_rotation,
        width=0.3
    )

    return gripper_meshes

In [22]:
# # 测试代码：传入一个4x4变换矩阵
# T = np.array([
#     [1, 0, 0, 0.1],  # 旋转矩阵和位移
#     [0, 1, 0, 0.2],
#     [0, 0, 1, 0.3],
#     [0, 0, 0, 1]
# ])

# # 调用生成抓手的函数
# gripper_meshes = generate_gripper_from_transform(T)

# # 可视化生成的抓手
# o3d.visualization.draw_geometries(gripper_meshes)

In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)
import matplotlib.pyplot as plt
import cv2

try:
    broadcaster
except NameError:
    broadcaster = TransformBroadcaster()           
    
    
def pointcloud_process(point_cloud, slice_tolerance=0.005):
    '''
        识别点云中的cubes
        returns:
            [
                json,
                T
            ]
    '''
    T = []
    remaining_pointcloud_count = 10000
    countdown = 80
    # 使用 open3d 可视化点云
    # o3d.visualization.draw_geometries([layer_point_cloud], window_name=f"Layer {layer} (Z range: {z_min:.4f} to {z_max:.4f})")
    movecount = 0
    while remaining_pointcloud_count > 50 and countdown > 0:
        cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=50000) 
        cube_point_cloud = filter_point_cloud_by_depth(cube_point_cloud, depth_threshold=-0.01)
        transform, remaining_pointcloud, deleted_pointcloud, fitness = register_and_filter(point_cloud,cube_point_cloud)
        remaining_pointcloud_count = len(remaining_pointcloud.points)
        # return transform, remaining_pointcloud, deleted_pointcloud
        if fitness > 0.01:
            countdown = countdown - 1
        # print(fitness)
        if check_transform_z_axis_alignment(transform) and fitness>0.70 and np.array_equal(transform, align_transform_z_axis(transform)):
        # if fitness > 0.70:
            movecount += 1
            print(movecount)
            

            broadcaster.update(transform)
            cube_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
                size=0.1,  # 坐标轴大小，可以根据需要调整
                origin=[0, 0, 0]  # 坐标轴的原点
            )
            theta = np.radians(45)  # 将角度转换为弧度
            transform_matrix_x_180 = np.array([
                [1, 0, 0, 0],
                [0, -1, 0, 0],
                [0, 0, -1, 0],
                [0, 0, 0, 1]
            ])
            graps_transform = transform @ transform_matrix_x_180
                        # o3d.visualization.draw_geometries([coordinate_frame, remaining_pointcloud], window_name="remaining_pointcloud")
            cube_point_cloud_transormed = copy.deepcopy(deleted_pointcloud)
            cube_point_cloud_transormed = cube_point_cloud_transormed.transform(np.linalg.inv(graps_transform))
            cube_point_cloud_transormed_cubes = enlarge_points_as_cubes(cube_point_cloud_transormed)

            cube_top_image = triangle_mesh_to_image(cube_point_cloud_transormed_cubes, image_size=(100, 100))
            cube_top_image = (cube_top_image / cube_top_image.max() * 255).astype(np.uint8)
            grasp_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
                size=0.1,  # 坐标轴大小，可以根据需要调整
                origin=[0, 0, 0]  # 坐标轴的原点
            )
            grasp_mesh = generate_gripper_from_transform(graps_transform)
            # 应用变换矩阵到坐标系
            grasp_coordinate_frame.transform(graps_transform)
            o3d.visualization.draw_geometries(grasp_mesh+[grasp_coordinate_frame, deleted_pointcloud, coordinate_frame], window_name="deleted_pointcloud")
            # o3d.visualization.draw_geometries([grasp_coordinate_frame, remaining_pointcloud, coordinate_frame], window_name="remaining_pointcloud")
            pick_rpy, pick_pos = matrix_to_rpy_and_translation(graps_transform)
            pick_pos_ = [a + b for a, b in zip(pick_pos, [0, 0, 0.10])]
            pick_rpy = [a + b for a, b in zip(pick_rpy, [0, 0, 0])]
            pick_place.move(pick_pos_, pick_rpy)
            # cube_top_image = point_cloud_to_image(cube_point_cloud_transormed)
            plt.imsave(f"test_{movecount}.png", cube_top_image)
            plt.show()
            point_cloud = remaining_pointcloud
            # TODO: 识别第一个面
            
            T.append(graps_transform)
        else:
            cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=50000)
            cube_point_cloud = filter_point_cloud_by_depth(cube_point_cloud, depth_threshold=-0.01)
    print(T)
            
Ts = pointcloud_process(filtered_point_cloud)

[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m


1


In [20]:
import logging
logging.basicConfig(level=logging.ERROR)
import matplotlib.pyplot as plt
import cv2


# 设置日志级别为 WARNING，这样 INFO 级别的日志就不会显示
logging.basicConfig(level=logging.ERROR)
try:
    broadcaster
except NameError:
    broadcaster = TransformBroadcaster()           
    

def pointcloud_splite_by_layers(point_cloud, max_layer, layer_thickness=0.042, slice_tolerance=0.005):
    """
    将点云切割成多层，每层0.04个高度，然后依次取每个方块放到固定位置并显示。
    """
    # 获取点和颜色信息直接使用 open3d 的方法
    points = point_cloud.points
    colors = point_cloud.colors if point_cloud.has_colors() else o3d.utility.Vector3dVector([[1, 1, 1]] * len(points))
    movecount = 0
    cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=50000) 
    cube_point_cloud = filter_point_cloud_by_depth(cube_point_cloud, depth_threshold=-0.01)
    
    for layer in range(max_layer-1, -1, -1):
        z_base = layer * layer_thickness
        z_min = z_base + slice_tolerance
        z_max = z_base + layer_thickness + slice_tolerance

        # 筛选属于该层的点
        mask = [z_min <= pt[2] <= z_max for pt in points]
        layer_points = [points[i] for i in range(len(points)) if mask[i]]
        layer_colors = [colors[i] for i in range(len(colors)) if mask[i]]

        if len(layer_points) > 0:
            print(f"Processing layer {layer} (Z range: {z_min:.4f} to {z_max:.4f})")
            
            # 创建一个新的点云对象用于显示16
            layer_point_cloud = o3d.geometry.PointCloud()
            
            layer_point_cloud.points = o3d.utility.Vector3dVector(layer_points)
            layer_point_cloud.colors = o3d.utility.Vector3dVector(layer_colors)
            o3d.visualization.draw_geometries([coordinate_frame, layer_point_cloud], window_name="layer_point_cloud")
            
            remaining_pointcloud_count = 10000
            countdown = 100
            # 使用 open3d 可视化点云
            # o3d.visualization.draw_geometries([layer_point_cloud], window_name=f"Layer {layer} (Z range: {z_min:.4f} to {z_max:.4f})")
            while remaining_pointcloud_count > 500 and countdown>0:
                transform, remaining_pointcloud, deleted_pointcloud, fitness = register_and_filter(layer_point_cloud, cube_point_cloud)
                remaining_pointcloud_count = len(remaining_pointcloud.points)
                # return transform, remaining_pointcloud, deleted_pointcloud
                if fitness > 0.1:
                    countdown = countdown - 1
                print(fitness)
                # if check_transform_z_axis_alignment(transform) and fitness>0.60 and  np.array_equal(transform, align_transform_z_axis(transform)):
                if fitness>0.60:
                    o3d.visualization.draw_geometries([coordinate_frame, remaining_pointcloud], window_name="remaining_pointcloud")
                    print("move ", movecount+1, " cube")
                    # 1. 矫正翻转的z轴
                    
                    # 2. 识别识别出的cube的top面是什么
                    cube_point_cloud_transormed = copy.deepcopy(deleted_pointcloud)
                    cube_point_cloud_transormed = cube_point_cloud_transormed.transform(np.linalg.inv(transform))
                    print(transform)
                    broadcaster.update(transform)
                    cube_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
                        size=0.1,  # 坐标轴大小，可以根据需要调整
                        origin=[0, 0, 0]  # 坐标轴的原点
                    )
                    theta = np.radians(45)  # 将角度转换为弧度
                    transform_matrix_x_180 = np.array([
                        [1, 0, 0, 0],
                        [0, -1, 0, 0],
                        [0, 0, -1, 0],
                        [0, 0, 0, 1]
                    ])
                    graps_transform = transform@transform_matrix_x_180
                    cube_point_cloud_transormed_cubes = enlarge_points_as_cubes(cube_point_cloud_transormed)
                    

                    cube_top_image = triangle_mesh_to_image(cube_point_cloud_transormed_cubes, image_size=(100, 100))
                    cube_top_image = (cube_top_image / cube_top_image.max() * 255).astype(np.uint8)
                    grasp_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
                        size=0.1,  # 坐标轴大小，可以根据需要调整
                        origin=[0, 0, 0]  # 坐标轴的原点
                    )

                    # 应用变换矩阵到坐标系
                    grasp_coordinate_frame.transform(graps_transform)
                    # o3d.visualization.draw_geometries([grasp_coordinate_frame, deleted_pointcloud, coordinate_frame], window_name="Filtered Point Cloud")
                    
                    # cube_top_image = point_cloud_to_image(cube_point_cloud_transormed)
                    plt.imsave(f"test_{movecount}.png", cube_top_image)
                    plt.show()
                    # o3d.visualization.draw_geometries([coordinate_frame, cube_coordinate_frame, deleted_pointcloud], window_name="Transformed Coordinate Frame")
                    
                    # print(gpt4o_analysis(f"test_{movecount}.png"))
                    # return
                    # image_recognizer_result = image_recognizer.recognize_image(cube_top_image)
                    # image_recognizer.display_results(cube_top_image, image_recognizer_result)
                    # return cube_top_image, image_recognizer_result
                    # image_recognizer_best_result = image_recognizer.selected_best_based_on_CNN(image_recognizer_result, cube_top_image)
                    # img, angle = image_recognizer.get_image_from_result(image_recognizer_best_result) # angle用于在place时候将其转正
                    # plt.imshow(img)
                    # plt.axis("off")
                    # plt.show()
                    angle = 0
                    
                    layer_point_cloud = remaining_pointcloud
                    
                    # continue
                    
                    # 3. move 
#                     # 如果匹配度高，则移动将其移动到指定位置
                    # 创建绕x轴旋转180度的4x4变换矩阵
                    
                    pick_rpy, pick_pos = matrix_to_rpy_and_translation(graps_transform)
                    pick_pos_ = [a + b for a, b in zip(pick_pos, [0, 0, 0.10])]
                    pick_rpy = [a + b for a, b in zip(pick_rpy, [0, 0, 0])]
                    # pick_place.move(pick_pos_, pick_rpy)
                    print("move_to", pick_pos_, pick_rpy)
            
                    pick_pos_ = [a + b for a, b in zip(pick_pos, [0.014, 0.020, 0.12])] # 安全距离
                    place_pos = [0.5-0.1*(movecount%4), 0.4-0.1*(movecount//4), 0.15]
                    place_rpy = [0, np.pi, -np.pi/4+angle]

                    
                    # 保持一个安全距离，或者做collision avoidance
                    # pick_place.move(pick_pos_, pick_rpy)
                    movecount = movecount+1
                else:
                    cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=50000) 
                    cube_point_cloud = filter_point_cloud_by_depth(cube_point_cloud, depth_threshold=-0.01)
                
            # 这里debug用，直接return
            # return transform, remaining_pointcloud, deleted_pointcloud
    print("done!") 
# T, remaining_pointcloud, deleted_pointcloud = pointcloud_splite_by_layers(filtered_point_cloud, max_layer)
# cube_top_image, image_recognizer_result = pointcloud_splite_by_layers(filtered_point_cloud, max_layer)
# cube_point_cloud_transormed = pointcloud_splite_by_layers(filtered_point_cloud, max_layer)
pointcloud_splite_by_layers(filtered_point_cloud, max_layer)

Processing layer 0 (Z range: 0.0050 to 0.0470)


[%t] [%l] %m
[%t] [%l] %m


0.6568554396423248
move  1  cube
[[-0.82532072 -0.5291985   0.19696358  0.56893383]
 [ 0.54097936 -0.84100484  0.00722457 -0.03136308]
 [ 0.16182409  0.11251582  0.98038419  0.02395822]
 [ 0.          0.          0.          1.        ]]
move_to [0.5689338307802249, -0.03136307750997802, 0.12395821807537309] [-3.027325521527317, -0.16253882482928028, 2.5613762806369786]
0.8390461997019374
move  2  cube
[[ 0.06259848 -0.99209167 -0.10879135  0.45976295]
 [ 0.99646799  0.06824087 -0.04893604  0.0824304 ]
 [ 0.05597305 -0.10534377  0.99285936  0.02386023]
 [ 0.          0.          0.          1.        ]]
move_to [0.45976294713341037, 0.08243039788916531, 0.12386022799508747] [3.0358867255183286, -0.056002320953226326, 1.5080584053879056]
0.7081159960258321
move  3  cube
[[-0.48858119  0.07856064  0.86897448  0.31169561]
 [ 0.87121089 -0.01057833  0.49079495 -0.14818237]
 [ 0.04774947  0.99685321 -0.06327451  0.02577682]
 [ 0.          0.          0.          1.        ]]


[%t] [%l] %m
[%t] [%l] %m


move_to [0.31169561373288673, -0.14818236614166583, 0.12577681878391994] [-1.5074071192414733, -0.04776762920723199, 2.0818987972479417]
0.0
0.851356799503799
move  4  cube
[[ 0.07212782  0.02087549 -0.99717691  0.5951346 ]
 [ 0.03152921  0.99923356  0.02319912 -0.27131852]
 [ 0.99689693 -0.0331135   0.07141435  0.02486218]
 [ 0.          0.          0.          1.        ]]


...

move_to [0.4517826578276561, -0.16509990668845834, 0.14181750008816782] [0.023767700164496818, -0.011170952259720757, -0.5135258121540024]
0.8276321910373702
move  6  cube
[[ 0.04307026  0.59629008 -0.80161281  0.47368501]
 [-0.05196555  0.80260834  0.59423853 -0.25292227]
 [ 0.99771967  0.01606224  0.06555509  0.02592202]
 [ 0.          0.          0.          1.        ]]
move_to [0.4736850136331662, -0.2529222700231813, 0.1259220175580822] [-2.901307467096101, -1.5032507839156157, -0.8787255998111483]
0.6904016126531245
move  7  cube
[[-0.03447521 -0.99887297  0.03262273  0.48596515]
 [-0.99893002  0.03544729  0.02970375 -0.10916899]
 [-0.03082666 -0.03156379 -0.99902625  0.04227949]
 [ 0.          0.          0.          1.        ]]


[%t] [%l] %m
[%t] [%l] %m


move_to [0.48596514868644547, -0.10916898606659527, 0.14227948552437925] [0.03158404431845031, 0.03083154601246796, -1.6052947676687943]
0.7089781361451388


KeyboardInterrupt: 

[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
[%t] [%l] %m
